First thing first: kaggle api key for downloading dataset

In [11]:
import json
import os

# Replace 'your_kaggle_username' and 'your_api_key' with your actual values
kaggle_credentials = {"username":"ghostysh","key":"2d9c751653ff10a0d920bed39841c7a1"}

# Save the credentials to the kaggle.json file
os.makedirs("/root/.kaggle/", exist_ok=True)
#/root/.config/kaggle/
with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_credentials, f)

# Set permissions for the file
os.chmod("/root/.kaggle/kaggle.json", 600)

print("kaggle.json created successfully!")

kaggle.json created successfully!


**Technical assignment 3**

**Santeri Hukari**


**Part 1: Dataset Preparation and Fine-Tuning (7 points)**

**Step 1: Download the IMDB Dataset (1 point)**
1. Use the IMDB dataset from Kaggle: /kaggle/input/imdb-dataset/IMDB
Dataset.csv.
2. Load the dataset using Pandas and verify it in your notebook.

In [12]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Move kaggle.json to the correct location
#!mkdir -p ~/.kaggle
!cp /root/.kaggle/kaggle.json /root/.config/kaggle/
#!cp /kaggle/working/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Initialize the Kaggle API
api = KaggleApi()
api.authenticate()

# Download the IMDB dataset
dataset_name = "lakshmi25npathi/imdb-dataset-of-50k-movie-reviews"
download_path = "/kaggle/working"

# Download and unzip the dataset
api.dataset_download_files(dataset_name, path=download_path, unzip=True)

# Verify the download
print("Downloaded files:", os.listdir(download_path))

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
Downloaded files: ['IMDB Dataset.csv', '.virtual_documents']


In [13]:
!ls /root/.config/kaggle/
!ls /kaggle/working

kaggle.json
'IMDB Dataset.csv'


In [14]:
import pandas as pd

# Path to the dataset
dataset_path = "/kaggle/working/IMDB Dataset.csv"

# Load the dataset
df = pd.read_csv(dataset_path)

# Display the first few rows
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


**Step 2: Data Preprocessing (1 point)**
1. Clean and preprocess the dataset:
  - Encode the sentiment column (positive -> 1, negative -> 0).
  - Retain only the review and label columns.
2. Split the data into training and validation, testing

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Dataset path
dataset_path = "/kaggle/working/IMDB Dataset.csv"

# Load the dataset
df = pd.read_csv(dataset_path)

# Encode the sentiment column: positive -> 1, negative -> 0
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Retain only the review and label columns
df = df[['review', 'sentiment']]

# Split the data into training, validation, and testing sets
train_data, temp_data = train_test_split(df, test_size=0.3, random_state=42, stratify=df['sentiment'])
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=temp_data['sentiment'])

# Print dataset sizes for verification
print("Training set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Testing set size:", len(test_data))

Training set size: 35000
Validation set size: 7500
Testing set size: 7500


**Step 3: Model Selection and Tokenization (1 point)**

**1. Select a pre-trained Hugging Face transformer model for fine-tuning (e.g., distilbert-base-uncased).**

**2. Tokenize the dataset with (see if required)**
- Truncation.
- Padding.
- Maximum sequence length of 256.

In [16]:
from transformers import AutoTokenizer

# Select the pre-trained model (e.g., DistilBERT)
model_name = "distilbert-base-uncased"

# Load the tokenizer for the selected model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset with truncation, padding, and max length
def tokenize_data(data):
    return tokenizer(
        data['review'].tolist(),
        truncation=True,
        padding=True,  # Add padding
        max_length=256,  # Set max sequence length
        return_tensors='pt'  # Return PyTorch tensors
    )

# Tokenize training, validation, and testing datasets
train_encodings = tokenize_data(train_data)
val_encodings = tokenize_data(val_data)
test_encodings = tokenize_data(test_data)

# Print a sample of tokenized data for verification
print("Tokenized Training Data:", train_encodings.input_ids.shape)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenized Training Data: torch.Size([35000, 256])


**Step 4: Fine-Tune the Model (2 points)**
1. Fine-tune the model on the IMDB dataset for 2 epochs using the Hugging Face
Trainer.
2. Set training parameters:
- Learning rate: 5e-5 or your own
- Batch size: 16 or 32
- Evaluation at the end of each epoch.

3. Ensure that metrics like accuracy, precision, recall, and F1-score are logged
during training.

In [18]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


In [20]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import evaluate
import numpy as np

import os
os.environ["WANDB_DISABLED"] = "true"

# Convert the dataframes to Hugging Face Dataset objects
def prepare_dataset(data, encodings):
    dataset = Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': data['sentiment'].tolist()
    })
    return dataset

train_dataset = prepare_dataset(train_data, train_encodings)
val_dataset = prepare_dataset(val_data, val_encodings)
test_dataset = prepare_dataset(test_data, test_encodings)

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Define metrics
metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="binary")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="binary")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="binary")
    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"],
    }

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    evaluation_strategy="epoch",    # Evaluate at the end of each epoch
    learning_rate=5e-5,             # Learning rate
    per_device_train_batch_size=16, # Batch size per device
    per_device_eval_batch_size=16,  # Evaluation batch size
    num_train_epochs=2,             # Number of epochs
    weight_decay=0.01,              # Weight decay for regularization
    logging_dir="./logs",           # Logging directory
    logging_steps=50,               # Log every 50 steps
    save_strategy="epoch",          # Save model at the end of each epoch
    save_total_limit=1,             # Keep only the latest checkpoint
    load_best_model_at_end=True,    # Load the best model at the end of training
    metric_for_best_model="accuracy" # Metric for selecting the best model
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-20-25560fd943dc>:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.250900,0.276810,0.905200,0.876954,0.942667,0.908624
2,0.109700,0.308114,0.915067,0.911011,0.920000,0.915484


TrainOutput(global_step=4376, training_loss=0.21126504272385968, metrics={'train_runtime': 945.8835, 'train_samples_per_second': 74.005, 'train_steps_per_second': 4.626, 'total_flos': 4636358952960000.0, 'train_loss': 0.21126504272385968, 'epoch': 2.0})

In [22]:
# Get the training metrics
training_metrics = trainer.state.log_history

# Filter the metrics to include only evaluation logs
eval_metrics = [entry for entry in training_metrics if "eval_loss" in entry]

# Convert to a Pandas DataFrame for better visualization
metrics_df = pd.DataFrame(eval_metrics)

# Select only relevant columns for display
columns_to_display = ["epoch", "eval_loss", "eval_accuracy", "eval_precision", "eval_recall", "eval_f1"]
metrics_df = metrics_df[columns_to_display]

# Display the table
print(metrics_df.to_string(index=False))

 epoch  eval_loss  eval_accuracy  eval_precision  eval_recall  eval_f1
   1.0   0.276810       0.905200        0.876954     0.942667 0.908624
   2.0   0.308114       0.915067        0.911011     0.920000 0.915484


**Step 5: Save and Upload the Model to Hugging Face (2 points)**
1. Save the fine-tuned model and tokenizer locally using save_pretrained().
2. Log in to Hugging Face using notebook_login.
3. Upload the model to Hugging Face using push_to_hub.
4. Verify the model on Hugging Face Hub and include the link in your notebook.

In [23]:
# Save the fine-tuned model and tokenizer locally
model_save_path = "./fine_tuned_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model saved at: {model_save_path}")

Model saved at: ./fine_tuned_model


In [25]:
from huggingface_hub import notebook_login

# Log in to Hugging Face
notebook_login()

In [27]:
!pip install huggingface_hub


In [29]:
# Push the model to the Hugging Face Hub
model_name_on_hub = "fine-tuned-imdb-model"
repo_url = model.push_to_hub(model_name_on_hub)

# Push the tokenizer to the same repo
tokenizer.push_to_hub(model_name_on_hub)

print(f"Model uploaded to Hugging Face Hub: {repo_url}")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model uploaded to Hugging Face Hub: https://huggingface.co/santerihukari/fine-tuned-imdb-model/commit/77e691894d2e9e742aab7772f2c755ab22aa33df


In case the output vanishes:

"Model uploaded to Hugging Face Hub: https://huggingface.co/santerihukari/fine-tuned-imdb-model/commit/77e691894d2e9e742aab7772f2c755ab22aa33df"

**Part 2: API Development and Testing (5 points)**


**Note:** from now on everything is done locally. All files are included in the github repository.

**Step 6: Set Up the Backend API (1 point)**
1. Use FastAPI or Flask, Express, Nest Nodejs to create an API.
2. Define a POST endpoint (/analyze/) that:

- Accepts:
  - text: The input text for sentiment analysis.
  - model: A parameter specifying the model to use (custom or llama).

- Returns:
  - Sentiment (positive or negative).
  - Confidence score.

```bash
(LLM3) santeri@LAPTOP-67I7VNKD:~$ curl -X 'POST' \
  'http://127.0.0.1:8000/analyze/' \
  -H 'Content-Type: application/json' \
  -d '{"text": "I love this movie!", "model": "custom"}'
{"sentiment":"positive","confidence":0.92798}

(LLM3) santeri@LAPTOP-67I7VNKD:~$ curl -X 'POST' \
  'http://127.0.0.1:8000/analyze/' \
  -H 'Content-Type: application/json' \
  -d '{"text": "I hate this movie!", "model": "custom"}'
{"sentiment":"negative","confidence":0.9756002426147461}
```

**Step 7: Load Models (1 point)**
1. Load the fine-tuned model from Hugging Face.
2. Access the Llama 3 model using the Groq Cloud API


**Step 8: Test the API Locally (1 point)**
1. Test the /analyze/ endpoint with both models (custom and llama) using:
- Postman.
- curl.
- Python requests.

Note: I assume this means only testing it with one of these. I could not get postman working. This could be clarified whether or not this means testing with all of them or only one.


curl
```bash
(LLM3) santeri@LAPTOP-67I7VNKD:~/LLM/LLM-tech3$ curl -X 'POST' 'http://127.0.0.1:8000/analyze/' -H 'Content-Type: application/json' -d '{"text": "The movie had stunning visuals and a captivating soundtrack, but the plot was so convoluted and poorly paced that it left me feeling both amazed and frustrated at the same time.", "model": "llama"}'
["{"sentiment":"negative","confidence":0.8214}"]

(LLM3) santeri@LAPTOP-67I7VNKD:~/LLM/LLM-tech3$ curl -X 'POST' 'http://127.0.0.1:8000/analyze/' -H 'Content-Type: application/json' -d '{"text": "The movie had stunning visuals and a captivating soundtrack, but the plot was so convoluted and poorly paced that it left me feeling both amazed and frustrated at the same time.", "model": "custom"}'
{"sentiment":"negative","confidence":0.9932}


**Step 9: Define the Llama 3 Prompt (1 point)**
1. Write a clear and reusable prompt for the Llama 3 model in Groq Cloud.
Example: can be improved more

> "Classify the sentiment of this text as positive or negative:'This movie was fantastic'"


>system_prompt = (
                "You are a highly accurate AI model specializing in binary sentiment analysis of text-based reviews. "
                "You will classify the given text as either 'positive' or 'negative', with no intermediate or neutral categories. "
                "Analyze the sentiment of the following text and respond strictly in the format: "
                "{\"sentiment\":\"negative\",\"confidence\":0.9982}. "
                "Do not include any additional explanation, comments, or formatting beyond this JSON structure. "
                "The confidence score must be a float between 0 and 1, rounded to four decimal places."
            )

**Step 10: Test with Both Models (1 point)**
1. Verify that the API works for both the fine-tuned model and the Llama 3 model.
2. Ensure the results return the sentiment score too.
3. For Groq you can add into prompt,
4. ?

Note: I don't understand parts 3 and 4 of this step. 3 seems like a sentence cut short, and 4. is just empty? Only completed 1 and 2.

**Part 3: UI Design and Explanation (3 points)**

**Step 11: React UI Design (1 point)**
- A text input field for user input.
- A dropdown menu for model selection:
  - Custom Model.
  - Llama 3.
- A button labeled "Analyze Sentiment" to send input and selected model to the backend API.
- A result display section showing:
  - Sentiment (positive or negative).
  - Confidence score(optional)


The code for the react app is in github repo

**Step 12: Submit GitHub Repository (1 point)**
1. Upload all code (notebook, backend, and UI explanation) to a public GitHub repository.
2. Include a README.md file that explains how to:
- Install dependencies.
- Run the notebook and API locally.
- Use the endpoints.

**Step 13: Record a YouTube Demo Video (1 point)**
1. Record a demo video (2-3 minutes) showing:
 - Testing the system with both models (custom and llama).
 - One question with custom fine and one with llam3 any llama 3 will be fine.

2. Upload the video to YouTube and include the link in your notebook.

Link: https://www.youtube.com/shorts/x5OkZ7pg7HY
The video is shorter. Testing the react api does not take 2-3 minutes.